In [1]:
import torch as t
from data import ChallengeDataset
from trainer import Trainer
from matplotlib import pyplot as plt
import numpy as np
import model
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score


c:\Users\hungd\OneDrive\FAU\StudOn\SS24\Deep Learning (DL)\Exercises\exercise4\src_to_implement\trainer.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
data = pd.read_csv('data.csv', sep=';')

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# set up data loading for the training and validation set each using t.utils.data.DataLoader and ChallengeDataset objects
train_dataset = ChallengeDataset(train_data)
val_dataset = ChallengeDataset(val_data)

train_loader = t.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = t.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# create an instance of our ResNet model
resnet = model.ResNet()

# set up a suitable loss criterion (you can find a pre-implemented loss functions in t.nn)
# set up the optimizer (see t.optim)
# create an object of type Trainer and set its early stopping criterion
criterion = t.nn.MSELoss()
optimizer = t.optim.Adam(resnet.parameters(), lr=0.001)
trainer = Trainer(resnet, criterion, optimizer, train_loader, val_loader, cuda=False, early_stopping_patience=5)
# trainer.set_early_stopping(patience=5)

In [3]:
_model = resnet
_optim = optimizer
_crit = criterion
_train_dl = train_loader
_val_test_dl = val_loader
train_losses, val_losses = [], []
epoch = 0

In [5]:
def train_step(x, y):
    # Reset the gradients
    _optim.zero_grad()

    # Propagate through the network
    outputs = _model(x)

    # Calculate the loss
    loss = _crit(outputs, y)

    # Compute gradient by backward propagation
    loss.backward()

    # Update weights
    _optim.step()

    # Return the loss
    return loss

def train_epoch():
    # Set the model to training mode
    _model.train()

    # Initialize variables for loss and number of samples
    total_loss = 0.0
    num_samples = 0

    # Iterate through the training set
    for x, y in _train_dl:

        # Perform a training step
        loss = train_step(x, y)

        # Update the total loss and number of samples
        total_loss += loss.item() * x.size(0)

        print("x.size(0): ", x.size(0))
        num_samples += x.size(0)
        print("Number of samples: ", num_samples)

    # Calculate the average loss for the epoch
    avg_loss = total_loss / num_samples

    # Return the average loss
    return avg_loss

In [ ]:
train_loss = train_epoch()

In [8]:
print("Train loss: ", train_loss)

Train loss:  0.09862016826868057


In [24]:
_model.eval()

# Disable gradient computation
with t.no_grad():
    # Initialize variables for loss and metrics
    total_loss = 0.0
    total_f1 = 0.0
    num_samples = 0

    # Iterate through the validation/test set
    for x, y in _val_test_dl:
        x = x.cpu()
        y = y.cpu()

        # Perform a validation/test step
        outputs = _model(x)
        loss = _crit(outputs, y)

        predictions = (outputs > 0.5).float()
        print("Predictions: ", predictions)

        # Update the total loss and number of samples
        total_loss += loss.item() * x.size(0)
        num_samples += x.size(0)

        # Calculate F1 score
        f1 = f1_score(y, predictions, average=None)
        print("F1 score: ", f1)
        total_f1 += f1 * x.size(0)

    # Calculate the average loss and F1 score
    avg_loss = total_loss / num_samples
    avg_f1 = total_f1 / num_samples



Predictions:  tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.]])
F1 score:  [0.36363636 0.        ]
Predictions:  tensor([[0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 1.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],


In [ ]:
_model.eval()

# Disable gradient computation
with t.no_grad():
    # Initialize variables for loss and metrics
    total_loss = 0.0
    total_f1 = 0.0
    num_samples = 0

    # Iterate through the validation/test set
    for x, y in _val_test_dl:
        x = x.cpu()
        y = y.cpu()

        # Perform a validation/test step
        outputs = _model(x)
        loss = _crit(outputs, y)
        

        # Update the total loss and number of samples
        total_loss += loss.item() * x.size(0)
        num_samples += x.size(0)

        # Calculate F1 score
        f1 = f1_score(y, outputs, average=None)
        total_f1 += f1 * x.size(0)

    # Calculate the average loss and F1 score
    avg_loss = total_loss / num_samples
    avg_f1 = total_f1 / num_samples



-----

In [ ]:
data = pd.read_csv('data.csv', sep=';')
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
print(train_data.shape, val_data.shape)

(1600, 3) (400, 3)


In [ ]:
train_dataset = ChallengeDataset(train_data)
val_dataset = ChallengeDataset(val_data)
print(len(train_dataset), len(val_dataset))

img, label = train_dataset[0]
print(img.shape, label)


1600 400
torch.Size([3, 224, 224]) tensor(0.)


In [ ]:
train_loader = t.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = t.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
print(len(train_loader), len(val_loader))
print(len(train_loader.dataset), len(val_loader.dataset))
print(train_loader.dataset.__getitem__(1)[0].shape)

50 13
1600 400
torch.Size([3, 224, 224])


In [ ]:
for x,y in train_loader:
    print(x.shape, y.shape)
    print(x, y)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32])
tensor([[[[-2.2048, -2.2048, -2.2537,  ..., -2.6937, -2.6937, -2.7426],
          [-2.2048, -2.2048, -2.2292,  ..., -2.6448, -2.6937, -2.7670],
          [-2.1803, -2.1559, -2.1559,  ..., -2.6203, -2.6937, -2.7670],
          ...,
          [-2.3026, -2.2781, -2.1315,  ..., -2.7670, -2.7914, -2.8159],
          [-2.3270, -2.3026, -2.2292,  ..., -2.7670, -2.7914, -2.7914],
          [-2.3270, -2.3515, -2.3026,  ..., -2.7426, -2.7914, -2.7914]],

         [[-2.2048, -2.2048, -2.2537,  ..., -2.6937, -2.6937, -2.7426],
          [-2.2048, -2.2048, -2.2292,  ..., -2.6448, -2.6937, -2.7670],
          [-2.1803, -2.1559, -2.1559,  ..., -2.6203, -2.6937, -2.7670],
          ...,
          [-2.3026, -2.2781, -2.1315,  ..., -2.7670, -2.7914, -2.8159],
          [-2.3270, -2.3026, -2.2292,  ..., -2.7670, -2.7914, -2.7914],
          [-2.3270, -2.3515, -2.3026,  ..., -2.7426, -2.7914, -2.7914]],

         [[-2.2048, -2.2048, -2.2537,  ..., -2.6937, -2

In [ ]:
model = model.ResNet()
for x,y in train_loader:
    print(model(x).shape)
    break


AttributeError: 'ResNet' object has no attribute 'ResNet'